In [2]:
import torch
import torch.nn.functional as F
import torch.cuda as cuda

torch.manual_seed(2022)
x = torch.ones(5)  # input tensor
y = torch.tensor([1,0,1],dtype=torch.float32)  # expected output
w = torch.randn(5, 3, requires_grad=True) # or through w.requires_grad_(True), set to True if need to compute the gradient w.r.t to such variable
b = torch.randn(3)
b.requires_grad_(True) # if is False, after running loss.backward(), we can not get the gradient for the parameter
z = torch.matmul(x, w)+b

# after defining the loss functions based on given parameter (matrix), we can then perform backprop to compute gradient
loss = F.binary_cross_entropy_with_logits(z, y)
b.requires_grad
# loss

/home/mcb/users/rdeng4/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [45]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for x@w = {(x@w).grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7fd1024439b0>
Gradient function for x@w = <SqueezeBackward3 object at 0x7fd102443f28>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x7fd102443390>


In [46]:
print(w.grad)
print(b.grad)
loss.backward() # after each .backward() call, autograd starts populating a new graph!
# after backward prop, we get gradients, they will accumulates (not overwrite!) to the existing grad of param
print(w.grad)
print(b.grad)

None
None
tensor([[-0.3062,  0.1600, -0.0679],
        [-0.3062,  0.1600, -0.0679],
        [-0.3062,  0.1600, -0.0679],
        [-0.3062,  0.1600, -0.0679],
        [-0.3062,  0.1600, -0.0679]])
tensor([-0.3062,  0.1600, -0.0679])


In [48]:
print(z.requires_grad)

# method 1 of drop grad:
with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)
z.requires_grad_(True) # attach grad_tracking back to tensor param
print(z.requires_grad)


# method 2 of drop grad: (BETTER!)
z_nograd = z.detach()
print(z_nograd.requires_grad)
print(z.requires_grad)


True
False
True


In [13]:
t1 = torch.rand(5,dtype=torch.float16)
t2 = torch.rand(5,dtype=torch.float16)

print(t1)
print(t2)
t3 = t1 == t2
t3 = t3.type(torch.float16)
t3.sum()

tensor([0.7080, 0.0303, 0.7388, 0.1738, 0.1694], dtype=torch.float16)
tensor([0.5781, 0.2246, 0.2480, 0.8145, 0.3691], dtype=torch.float16)


tensor(0., dtype=torch.float16)